In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [2]:
spark

In [3]:
import pandas as pd
import numpy as np
from pyspark.sql.functions import *

In [4]:
np.random.seed(456)

pandas_dataframe = pd.DataFrame(
    dict(n=np.arange(20), group=np.random.choice(list("abc"), 20))
)
pandas_dataframe

,n,group
0,0,b
1,1,b
2,2,c
3,3,a
4,4,c
5,5,c
6,6,a
7,7,b
8,8,a
9,9,b


In [5]:
df = spark.createDataFrame(pandas_dataframe)
df

DataFrame[n: bigint, group: string]

In [6]:
df.show(5)

+---+-----+
|  n|group|
+---+-----+
|  0|    b|
|  1|    b|
|  2|    c|
|  3|    a|
|  4|    c|
+---+-----+
only showing top 5 rows



In [7]:
from vega_datasets import data

In [8]:
from pyspark.sql.functions import month, year, quarter

In [9]:
import seaborn as sns
df = sns.load_dataset('tips')

In [10]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [11]:
df = sns.load_dataset('mpg')

In [12]:
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger


1. Create a spark data frame that contains your favorite programming languages.

In [13]:
languages = pd.DataFrame({'languages' : ['Python', 'Javascript', 'SQL', 'Scala', 'Ruby', 'Java', 'C#']})

spark = pyspark.sql.SparkSession.builder.getOrCreate()
df = spark.createDataFrame(languages)
df.show()

+----------+
| languages|
+----------+
|    Python|
|Javascript|
|       SQL|
|     Scala|
|      Ruby|
|      Java|
|        C#|
+----------+



In [14]:
df.printSchema()

root
 |-- languages: string (nullable = true)



In [15]:
print((df.count(), len(df.columns)))

(7, 1)


In [16]:
df.show(5)

+----------+
| languages|
+----------+
|    Python|
|Javascript|
|       SQL|
|     Scala|
|      Ruby|
+----------+
only showing top 5 rows



3. Load the tips dataset as a spark dataframe.

 - What percentage of observations are smokers?
 - Create a column that contains the tip percentage
 - Calculate the average tip percentage for each combination of sex and smoker.

In [17]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [18]:
df = spark.createDataFrame(df)
df.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [19]:
# What percentage of observations are smokers?
(df.filter(df.smoker == 'Yes').count()) / df.count()

0.38114754098360654

In [20]:
# Create a column that contains the tip percentage
df = df.withColumn('tip_percent', round((col('tip')/col('total_bill')*100), 2))
df.show(10)

+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percent|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|       5.94|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|      16.05|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|      16.66|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|      13.98|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|      14.68|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|      18.62|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|      22.81|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|      11.61|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|      13.03|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|      21.85|
+----------+----+------+------+---+------+----+-----------+
only showing top 10 rows



In [21]:
# Calculate the average tip percentage for each combination of sex and smoker.
df.groupBy('sex', 'smoker').agg(round(mean(col('tip_percent')),2).alias("avg_tip_percent")).show()

+------+------+---------------+
|   sex|smoker|avg_tip_percent|
+------+------+---------------+
|  Male|    No|          16.07|
|  Male|   Yes|          15.28|
|Female|    No|          15.69|
|Female|   Yes|          18.21|
+------+------+---------------+



4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

 - Convert the temperatures to farenheight.
 - Which month has the most rain, on average?
 - Which year was the windiest?
 - What is the most frequent type of weather in January?
 - What is the average high and low tempurature on sunny days in July in 2013 and 2014?
 - What percentage of days were rainy in q3 of 2015?
 - For each year, find what percentage of days it rained (had non-zero precipitation).

In [22]:
weather = data.seattle_weather()
df = spark.createDataFrame(weather)
df.show(5)

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [23]:
# Convert the temperatures to farenheight.
df.withColumn('temp_max', df.temp_max * 9 / 5 + 32).withColumn('temp_min', df.temp_min * 9 / 5 + 32).show(10)

+-------------------+-------------+------------------+--------+----+-------+
|               date|precipitation|          temp_max|temp_min|wind|weather|
+-------------------+-------------+------------------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|             55.04|    41.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|             51.08|   37.04| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|             53.06|   44.96| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|             53.96|   42.08| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|             48.02|   37.04| 6.1|   rain|
|2012-01-06 00:00:00|          2.5|             39.92|   35.96| 2.2|   rain|
|2012-01-07 00:00:00|          0.0|             44.96|   37.04| 2.3|   rain|
|2012-01-08 00:00:00|          0.0|              50.0|   37.04| 2.0|    sun|
|2012-01-09 00:00:00|          4.3|             48.92|    41.0| 3.4|   rain|
|2012-01-10 00:00:00|          1.0|42.980000000000004|   33.08| 3.4|   rain|

In [24]:
# Which month has the most rain, on average?
(df
 .withColumn('month', month('date'))
 .groupBy('month')
 .agg(mean('precipitation').alias('avg_monthly_rain'))
 .sort('avg_monthly_rain')
 .show())

+-----+-------------------+
|month|   avg_monthly_rain|
+-----+-------------------+
|    7|0.38870967741935486|
|    6| 1.1075000000000002|
|    8| 1.3201612903225806|
|    5| 1.6733870967741935|
|    9| 1.9624999999999997|
|    4|  3.128333333333333|
|    2|  3.734513274336283|
|    1| 3.7580645161290316|
|   10|  4.059677419354839|
|    3|  4.888709677419355|
|   12|  5.021774193548388|
|   11|  5.354166666666667|
+-----+-------------------+



In [25]:
# Which year is the windiest?
(df
 .withColumn('year', year('date'))
 .groupBy('year')
 .agg(sum('wind'))
 .show())

+----+------------------+
|year|         sum(wind)|
+----+------------------+
|2015|            1153.3|
|2013|1100.8000000000002|
|2014|1236.5000000000005|
|2012|1244.6999999999998|
+----+------------------+



In [26]:
# What is the most frequent type of weather in January?
(df
 .withColumn('month', month('date'))
 .filter(expr("month == 1"))
 .groupBy('weather')
 .count()
 .sort(col('count').desc())
 .show())

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



In [27]:
# What is the average high and low tempurature on sunny days in July in 2013 and 2014?
(df
 .filter(month('date') == 7)
 .filter(year('date') > 2012)
 .filter(year('date') < 2015)
 .filter(col('weather') == lit('sun'))
 .agg(avg('temp_max').alias('average_high_temp'), avg('temp_min').alias('average_low_temp'))
 .show()
)

+------------------+-----------------+
| average_high_temp| average_low_temp|
+------------------+-----------------+
|26.828846153846158|14.18269230769231|
+------------------+-----------------+



In [28]:
# What percentage of days were rainy in q3 of 2015?
(df
 .withColumn('quarter', quarter('date'))
 .withColumn('year', year('date'))
 .filter(expr("year = 2015"))
 .filter(expr("quarter = 3"))
 .select(when(col('precipitation') > 0, 1).otherwise(0).alias('was_rainy'))
 .agg(mean('was_rainy'))
 .show())

+-------------------+
|     avg(was_rainy)|
+-------------------+
|0.18478260869565216|
+-------------------+



In [29]:
# For each year, find what percentage of days it rained (had non-zero precipitation)
(df
 .withColumn('rain', (when(col('precipitation') > 0, 1).otherwise(0)))
 .groupBy(year('date').alias('year'))
 .agg(mean(col('rain')).alias('pct_days_with_rain'))
 .show()
)

+----+-------------------+
|year| pct_days_with_rain|
+----+-------------------+
|2015|0.39452054794520547|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2012|0.48360655737704916|
+----+-------------------+

